In [1]:
import pandas as pd

In [24]:
df = pd.read_csv('../data/reddit_since10.csv')
df.head()

FileNotFoundError: [Errno 2] File b'../data/reddit_since10.csv' does not exist: b'../data/reddit_since10.csv'

**Training and Using our Model**

In [1]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
from joblib import dump, load
import pandas as pd
import numpy as np
import itertools
import pickle
import sys
import os

np.set_printoptions(threshold=sys.maxsize)

In [23]:
print("BUILDING...")
bitcoin = pd.read_csv("../../sentiment/sentiment_labels.csv")
reddit = pd.read_csv("../../sentiment/reddit_text.csv")
twitter = pd.read_csv("../../sentiment/twitter_text.csv")
bitcoin['datetime'] = pd.to_datetime(bitcoin['datetime'], errors='coerce')
reddit['datetime'] = pd.to_datetime(reddit['datetime'], errors='coerce')
twitter['datetime'] = pd.to_datetime(twitter['datetime'], errors='coerce')
bitcoin['datetime'] = pd.to_datetime(bitcoin["datetime"].dt.strftime('%Y-%m-%d'))
reddit['datetime'] = pd.to_datetime(reddit["datetime"].dt.strftime('%Y-%m-%d'))
twitter['datetime'] = pd.to_datetime(twitter["datetime"].dt.strftime('%Y-%m-%d'))
merged1 = pd.merge(bitcoin, reddit, on='datetime')
merged2 = pd.merge(bitcoin, twitter, on='datetime')
merged1 = merged1.drop("datetime",1)
merged2 = merged2.drop("datetime",1)
merged1.columns = ("analysis", "text")
merged2.columns = ("analysis", "text")
data_reddit = merged1
data_twitter = merged2   
data_both = merged1.append(merged2)
text_data, data_labels = [],[]
index_iteration = 0
for index, row in data_both.iterrows():
    text_data.append(str(row['text']))
    data_labels.append(row['analysis'])
if len(text_data) == len(data_labels): pass
else: print("LENGTH INVALID!\n'text_data' HAS LENGTH:", len(text_data), "\n'data_labels' HAS LENGTH:", len(data_labels))
print("TRAINING...")
solver_algorithm = "liblinear"
vectorizer = CountVectorizer(analyzer = 'word', lowercase = False)
features = vectorizer.fit_transform(text_data)
features_nd = features.toarray()
X_train, X_test, y_train, y_test  = train_test_split(features_nd, data_labels, train_size=0.80)
model = LogisticRegression(solver = solver_algorithm)
model = model.fit(X=X_train, y=y_train)
y_pred = model.predict(X_test)
acc = accuracy_score(y_test, y_pred)
y_pred = model.predict(X_test)
print("DONE → ACCURACY SCORE:",str(round(acc * 100, 3)) + "%")

BUILDING...
TRAINING...
DONE → ACCURACY SCORE: 61.726%


In [141]:
def get_our_sentiment(text):
    features = vectorizer.transform([text])
    features_nd = features.toarray()
    out = sent_model.predict(features_nd)
    return out[0]

In [148]:
df['title_polarity']= df['title'].apply(get_our_sentiment)

In [149]:
df.head()

,title,score,id,comments,created,body,title_polarity
0,New York,1,chkuap,0,1564042244,Defending the status of a financial innovation...,1.0
1,Dappstats will be the first project of crypto ...,1,chkud7,0,1564042264,[removed],0.0
2,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuis,0,1564042295,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0
3,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuu9,1,1564042363,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0
4,Historical tick-by-tick cryptocurrency market ...,1,chkuw2,0,1564042376,"Hi, if anyone here is in need of tick by tick ...",0.0


In [150]:
df['body_polarity']= df['body'].fillna(value='').apply(get_our_sentiment)

In [151]:
df.head()

,title,score,id,comments,created,body,title_polarity,body_polarity
0,New York,1,chkuap,0,1564042244,Defending the status of a financial innovation...,1.0,0.0
1,Dappstats will be the first project of crypto ...,1,chkud7,0,1564042264,[removed],0.0,1.0
2,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuis,0,1564042295,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0,0.0
3,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuu9,1,1564042363,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",1.0,0.0
4,Historical tick-by-tick cryptocurrency market ...,1,chkuw2,0,1564042376,"Hi, if anyone here is in need of tick by tick ...",0.0,1.0


In [ ]:
df.to_csv('../data/reddit_sent_ours.csv',index=False)

**Using textblob**

In [ ]:
pip install -U textblob

In [4]:
from textblob import TextBlob as tb

def get_sentiment(text):
    blob = tb(text)
    return tuple(blob.sentiment)

In [ ]:
df = pd.read_csv('reddit_since10.csv')

In [30]:
df['title_polarity'], df['title_subjectivity'] = df['title'].apply(get_sentiment).apply(pd.Series).values.T

In [7]:
df.head(10)

,title,score,id,comments,created,body,title_polarity,title_subjectivity
0,New York,1,chkuap,0,1564042244,Defending the status of a financial innovation...,0.136364,0.454545
1,Dappstats will be the first project of crypto ...,1,chkud7,0,1564042264,[removed],0.361111,0.555556
2,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuis,0,1564042295,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000
3,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuu9,1,1564042363,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000
4,Historical tick-by-tick cryptocurrency market ...,1,chkuw2,0,1564042376,"Hi, if anyone here is in need of tick by tick ...",0.250000,0.250000
5,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkv1t,0,1564042414,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000
6,XR Token at Alterdice Cryptocurrency Exchange!🔥🔥,1,chkv5s,0,1564042436,NaN,0.000000,0.000000
7,Daimler and Riddle &amp; Code create a cool wa...,1,chkvhl,0,1564042507,The German automaker Daimler and blockchain-co...,0.350000,0.650000
8,A great company headed by a team of specialist...,1,chkwml,0,1564042760,A great company headed by a team of specialis...,0.600000,0.625000
9,I recommend to pay Your attention to #ICO a ne...,1,chkwzy,0,1564042838,I recommend to pay Your attention to #ICO a n...,0.136364,0.454545


In [8]:
df['body'] = df['body'].fillna(value='')

In [10]:
df['body_polarity'], df['body_subjectivity'] = df['body'].apply(get_sentiment).apply(pd.Series).values.T

In [11]:
df.head(10)

,title,score,id,comments,created,body,title_polarity,title_subjectivity,body_polarity,body_subjectivity
0,New York,1,chkuap,0,1564042244,Defending the status of a financial innovation...,0.136364,0.454545,0.051399,0.363636
1,Dappstats will be the first project of crypto ...,1,chkud7,0,1564042264,[removed],0.361111,0.555556,0.000000,0.000000
2,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuis,0,1564042295,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000,0.066667,0.183333
3,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkuu9,1,1564042363,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000,0.066667,0.183333
4,Historical tick-by-tick cryptocurrency market ...,1,chkuw2,0,1564042376,"Hi, if anyone here is in need of tick by tick ...",0.250000,0.250000,0.219286,0.355714
5,Interview with BitBay Pay CEO — Patryk Kadlec,1,chkv1t,0,1564042414,"Patryk Kadlec is the CEO of BitBay Pay, a lice...",0.000000,0.000000,0.066667,0.183333
6,XR Token at Alterdice Cryptocurrency Exchange!🔥🔥,1,chkv5s,0,1564042436,,0.000000,0.000000,0.000000,0.000000
7,Daimler and Riddle &amp; Code create a cool wa...,1,chkvhl,0,1564042507,The German automaker Daimler and blockchain-co...,0.350000,0.650000,0.102783,0.485410
8,A great company headed by a team of specialist...,1,chkwml,0,1564042760,A great company headed by a team of specialis...,0.600000,0.625000,0.600000,0.625000
9,I recommend to pay Your attention to #ICO a ne...,1,chkwzy,0,1564042838,I recommend to pay Your attention to #ICO a n...,0.136364,0.454545,0.284848,0.643939


In [152]:
df.to_csv('../data/reddit_sent.csv',index=False)